# Práctica 2: Transformaciones geométricas

### 1a. Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente.

#### Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.

#### Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.

#### Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de escala.

In [7]:
import cv2 as cv
import numpy as np

# ======================================
# CARGA DE IMAGEN
# ======================================
img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

h, w = img.shape[:2]
size = (w, h)

def nothing(x):
    pass

# ======================================
# CREAR UNA SOLA VENTANA CON TODOS LOS CONTROLES
# ======================================
cv.namedWindow("Transformaciones")

# --- Traslación ---
cv.createTrackbar('tx', 'Transformaciones', w, 2 * w, nothing)
cv.createTrackbar('ty', 'Transformaciones', h, 2 * h, nothing)

# --- Rotación ---
cv.createTrackbar('Angulo', 'Transformaciones', 0, 360, nothing)
cv.createTrackbar('Centro X', 'Transformaciones', w // 2, w, nothing)
cv.createTrackbar('Centro Y', 'Transformaciones', h // 2, h, nothing)

# --- Escalado ---
cv.createTrackbar('Uniforme', 'Transformaciones', 0, 1, nothing)
cv.createTrackbar('Escalado X', 'Transformaciones', 100, 300, nothing)
cv.setTrackbarMin('Escalado X', 'Transformaciones', 10)
cv.createTrackbar('Escalado Y', 'Transformaciones', 100, 300, nothing)
cv.setTrackbarMin('Escalado Y', 'Transformaciones', 10)

# ======================================
# BUCLE PRINCIPAL
# ======================================
while True:
    # ----- Lectura de trackbars -----
    tx = cv.getTrackbarPos('tx', 'Transformaciones') - w
    ty = cv.getTrackbarPos('ty', 'Transformaciones') - h
    ang = cv.getTrackbarPos('Angulo', 'Transformaciones')
    cx = cv.getTrackbarPos('Centro X', 'Transformaciones')
    cy = cv.getTrackbarPos('Centro Y', 'Transformaciones')
    sx = cv.getTrackbarPos('Escalado X', 'Transformaciones') / 100
    sy = cv.getTrackbarPos('Escalado Y', 'Transformaciones') / 100
    uniforme = cv.getTrackbarPos('Uniforme', 'Transformaciones')

    # Escalado uniforme (si está activado)
    if uniforme == 1:
        sx = sy

    # ======================================
    # COMBINACIÓN DE TRANSFORMACIONES
    # ======================================

    # 1️⃣ Matriz de rotación (incluye el centro)
    R = cv.getRotationMatrix2D((cx, cy), ang, 1.0)

    # 2️⃣ Aplicar rotación a la imagen original
    rotated = cv.warpAffine(img, R, size)

    # 3️⃣ Matriz de escalado
    S = np.float32([[sx, 0, 0],
                    [0, sy, 0]])

    scaled = cv.warpAffine(rotated, S, size)

    # 4️⃣ Matriz de traslación
    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])

    # 5️⃣ Aplicar traslación al resultado final
    final = cv.warpAffine(scaled, T, size)


    tipo = " (UNIFORME)" if uniforme == 1 else " (NO UNIFORME)"
    info = f"Ang={ang}° | C=({cx},{cy}) | Tx={tx:+d}, Ty={ty:+d} | Sx={sx:.2f}, Sy={sy:.2f}{tipo}"
    #cv.putText(final, info, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv.imshow("Transformaciones", final)

    # Salir con ESC
    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()
cv.waitKey(1)


-1

### 1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen.

In [1]:
# Proyección de una imagen en otra mediante homografía

import cv2 as cv
import numpy as np

# ------------------------------
# Cargar imagen original
# ------------------------------
img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

h, w = img.shape[:2]
img2 = np.ones((h, w, 3), dtype=np.uint8) * 255  # segunda ventana en blanco

# Copias para dibujar los puntos
clone1 = img.copy()
clone2 = img2.copy()

# Listas para guardar puntos
pts_src = []
pts_dst = []

# ------------------------------
# Callback del ratón para ambas ventanas
# ------------------------------
def seleccionar_puntos(event, x, y, flags, param):
    global pts_src, pts_dst, clone1, clone2

    # param = 1 → primera imagen (fuente)
    # param = 2 → segunda imagen (destino)
    if event == cv.EVENT_LBUTTONDOWN:
        if param == 1 and len(pts_src) < 4:
            pts_src.append((x, y))
            cv.circle(clone1, (x, y), 5, (0, 0, 255), -1)
            cv.putText(clone1, f"{len(pts_src)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv.imshow('Original', clone1)

        elif param == 2 and len(pts_dst) < 4:
            pts_dst.append((x, y))
            cv.circle(clone2, (x, y), 5, (255, 0, 0), -1)
            cv.putText(clone2, f"{len(pts_dst)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            cv.imshow('Destino', clone2)

        # Si ya hay 4 puntos en cada imagen, proyectar
        if len(pts_src) == 4 and len(pts_dst) == 4:
            proyectar_imagen()

# ------------------------------
# Función para realizar la proyección
# ------------------------------
def proyectar_imagen():
    global pts_src, pts_dst, img, img2

    # Calcular matriz de transformación de perspectiva
    M = cv.getPerspectiveTransform(np.float32(pts_src), np.float32(pts_dst))

    # Aplicar transformación a la imagen original
    proyectada = cv.warpPerspective(img, M, (img2.shape[1], img2.shape[0]))

    # Mostrar la proyección
    cv.imshow('Proyeccion', proyectada)
    print("Transformación completada.")
    cv.waitKey(0)
    cv.destroyAllWindows()

# ------------------------------
# Ventanas e interacción
# ------------------------------
cv.namedWindow('Original')
cv.namedWindow('Destino')

cv.setMouseCallback('Original', seleccionar_puntos, 1)
cv.setMouseCallback('Destino', seleccionar_puntos, 2)

print("Haz clic en 4 puntos en la imagen ORIGINAL (rojo).")
print("Luego haz clic en 4 puntos en la imagen DESTINO (azul).")

cv.imshow('Original', img)
cv.imshow('Destino', img2)


while True:
    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()
cv.waitKey(1)

# TODO : Hacer que la tecla r resete la configuración
#     - Cambiar forma de seleccionar punto de rotación
#     - Cambiar forma de hacer el escalado uniforme

Haz clic en 4 puntos en la imagen ORIGINAL (rojo).
Luego haz clic en 4 puntos en la imagen DESTINO (azul).
Transformación completada.


KeyboardInterrupt: 

### 1c. Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz

In [ ]:
import cv2
import numpy as np

img = cv.imread('images/eii.png')
size = (img.shape[1], img.shape[0])
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

def nothing(x):
    pass

def apply_distortion(image, k1, k2):
    h, w = image.shape[:2]
    
    distCoeff = np.zeros((4,1), np.float64)
    distCoeff[0,0] = k1  
    distCoeff[1,0] = k2  

    # Suponer matriz unitaria para la cámara
    cam = np.eye(3, dtype=np.float32)
    cam[0,2] = w/2.0  # Definir centro x
    cam[1,2] = h/2.0  # Definir centro y
    cam[0,0] = 10.    # Definir longitud focal x
    cam[1,1] = 10.    # Definir longitud focal y

    # Aplicar distorsión a la imagen
    distorted_img = cv2.undistort(image, cam, distCoeff)

    return distorted_img

h, w = img.shape[:2]

cv2.namedWindow("Distorsion")

cv2.createTrackbar('Barril/Almohadilla', 'Distorsion', 0, 1, nothing)
cv2.createTrackbar('K1', 'Distorsion', 0, 10, nothing)
cv2.createTrackbar('K2', 'Distorsion', 0, 10, nothing)

while True:
    modo = cv2.getTrackbarPos('Barril/Almohadilla', 'Distorsion')
    k1 = cv2.getTrackbarPos('K1', 'Distorsion')
    k2 = cv2.getTrackbarPos('K2', 'Distorsion')
    
    if modo == 0:
        k1 = k1 / 100000
        k2 = k2 / 100000
    else:
        k1 = k1 / -100000
        k2 = k2 / -100000
    
    distorsion_img = apply_distortion(img, k1, k2)
    
    
    cv2.imshow("Distorsion", distorsion_img)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
cv2.waitKey(1)


-1

### (Optativo – 5 puntos)
- Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en
una imagen destino y realizar la transformación afín.
- Calcular la imagen especular a partir de una imagen.
- Tratar una recta que será el eje de reflexión y “reflejar” la imagen.
- Otras aportaciones

- Hacer la parte obligatoria sobre vídeo en lugar de sobre imagen.

In [ ]:
# Transformaciones combinadas (Translación, Rotación, Escalado)

import cv2 as cv
import numpy as np

# ======================================
# CARGA DE VIDEO O CÁMARA
# ======================================
# Si quieres usar la cámara, pon 0
# Si prefieres un archivo, cambia por la ruta: 'videos/video.mp4'
cap = cv.VideoCapture('videos/times-square.mp4')

if not cap.isOpened():
    print("Error: no se pudo abrir el vídeo o la cámara")
    exit()

# Obtener dimensiones del primer frame
ret, frame = cap.read()
if not ret:
    print("Error: no se pudo leer el primer frame")
    cap.release()
    exit()

h, w = frame.shape[:2]
size = (w, h)

def nothing(x):
    pass

# ======================================
# CREAR UNA SOLA VENTANA CON TODOS LOS CONTROLES
# ======================================
cv.namedWindow("Transformaciones")

# --- Traslación ---
cv.createTrackbar('tx', 'Transformaciones', w, 2 * w, nothing)
cv.createTrackbar('ty', 'Transformaciones', h, 2 * h, nothing)

# --- Rotación ---
cv.createTrackbar('Angulo', 'Transformaciones', 0, 360, nothing)
cv.createTrackbar('Centro X', 'Transformaciones', w // 2, w, nothing)
cv.createTrackbar('Centro Y', 'Transformaciones', h // 2, h, nothing)

# --- Escalado ---
cv.createTrackbar('Uniforme', 'Transformaciones', 0, 1, nothing)
cv.createTrackbar('Escalado X', 'Transformaciones', 100, 300, nothing)
cv.setTrackbarMin('Escalado X', 'Transformaciones', 10)
cv.createTrackbar('Escalado Y', 'Transformaciones', 100, 300, nothing)
cv.setTrackbarMin('Escalado Y', 'Transformaciones', 10)

# ======================================
# BUCLE PRINCIPAL
# ======================================
while True:
    ret, frame = cap.read()
    if not ret:
        print("Fin del vídeo o error al leer frame.")
        break

    # ----- Lectura de trackbars -----
    tx = cv.getTrackbarPos('tx', 'Transformaciones') - w
    ty = cv.getTrackbarPos('ty', 'Transformaciones') - h
    ang = cv.getTrackbarPos('Angulo', 'Transformaciones')
    cx = cv.getTrackbarPos('Centro X', 'Transformaciones')
    cy = cv.getTrackbarPos('Centro Y', 'Transformaciones')
    sx = cv.getTrackbarPos('Escalado X', 'Transformaciones') / 100
    sy = cv.getTrackbarPos('Escalado Y', 'Transformaciones') / 100
    uniforme = cv.getTrackbarPos('Uniforme', 'Transformaciones')

    # Escalado uniforme (si está activado)
    if uniforme == 1:
        sx = sy

    # ======================================
    # COMBINACIÓN DE TRANSFORMACIONES
    # ======================================

    # 1️⃣ Rotación
    R = cv.getRotationMatrix2D((cx, cy), ang, 1.0)
    rotated = cv.warpAffine(frame, R, size)

    # 2️⃣ Escalado
    S = np.float32([[sx, 0, 0],
                    [0, sy, 0]])
    scaled = cv.warpAffine(rotated, S, size)

    # 3️⃣ Traslación
    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])
    final = cv.warpAffine(scaled, T, size)

    # ======================================
    # INFORMACIÓN EN PANTALLA
    # ======================================
    tipo = " (UNIFORME)" if uniforme == 1 else " (NO UNIFORME)"
    info = f"Ang={ang}° | C=({cx},{cy}) | Tx={tx:+d}, Ty={ty:+d} | Sx={sx:.2f}, Sy={sy:.2f}{tipo}"
    cv.putText(final, info, (10, 30),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Mostrar vídeo transformado
    cv.imshow("Transformaciones", final)

    # Salir con ESC
    if cv.waitKey(10) & 0xFF == 27:
        break

# ======================================
# LIBERAR RECURSOS
# ======================================
cap.release()
cv.destroyAllWindows()
cv.waitKey(1)


-1

In [5]:
# Trazar una ventana de proyección y proyectar el vídeo.

import cv2 as cv
import numpy as np

# ------------------------------
# Cargar el vídeo
# ------------------------------
# Puedes usar 0 para la cámara o un archivo de vídeo
cap = cv.VideoCapture('videos/times-square.mp4')
if not cap.isOpened():
    print("Error: no se pudo cargar el vídeo")
    exit()

# Leer primer fotograma para conocer tamaño
ret, frame = cap.read()
if not ret:
    print("Error: no se pudo leer el primer fotograma del vídeo")
    exit()

h, w = frame.shape[:2]
img2 = np.ones((h, w, 3), dtype=np.uint8) * 255  # ventana destino en blanco

# Copias para dibujar los puntos
clone1 = frame.copy()
clone2 = img2.copy()

# Listas para guardar puntos
pts_src = []
pts_dst = []

M = None  # matriz de homografía (se calculará una sola vez)

# ------------------------------
# Callback del ratón para ambas ventanas
# ------------------------------
def seleccionar_puntos(event, x, y, flags, param):
    global pts_src, pts_dst, clone1, clone2, M

    if event == cv.EVENT_LBUTTONDOWN:
        # param = 1 → primera ventana (fuente)
        # param = 2 → segunda ventana (destino)
        if param == 1 and len(pts_src) < 4:
            pts_src.append((x, y))
            cv.circle(clone1, (x, y), 5, (0, 0, 255), -1)
            cv.putText(clone1, f"{len(pts_src)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv.imshow('Fuente (Video)', clone1)

        elif param == 2 and len(pts_dst) < 4:
            pts_dst.append((x, y))
            cv.circle(clone2, (x, y), 5, (255, 0, 0), -1)
            cv.putText(clone2, f"{len(pts_dst)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            cv.imshow('Destino', clone2)

        # Cuando hay 4 puntos en cada imagen, calcular homografía
        if len(pts_src) == 4 and len(pts_dst) == 4:
            M = cv.getPerspectiveTransform(np.float32(pts_src), np.float32(pts_dst))
            print("Homografía calculada. Reproduciendo vídeo proyectado...")
            cv.destroyWindow('Fuente (Video)')
            cv.destroyWindow('Destino')
            reproducir_video()

# ------------------------------
# Reproducir el vídeo proyectado
# ------------------------------
def reproducir_video():
    global cap, M, h, w, img2

    # Volver a inicio del vídeo
    cap.set(cv.CAP_PROP_POS_FRAMES, 0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Aplicar homografía al frame
        proyectado = cv.warpPerspective(frame, M, (w, h))

        # Mostrar
        cv.imshow('Proyeccion del Video', proyectado)

        # Salir con ESC
        if cv.waitKey(30) & 0xFF == 27:
            break

    cap.release()
    cv.destroyAllWindows()

# ------------------------------
# Interfaz inicial
# ------------------------------
cv.namedWindow('Fuente (Video)')
cv.namedWindow('Destino')

cv.setMouseCallback('Fuente (Video)', seleccionar_puntos, 1)
cv.setMouseCallback('Destino', seleccionar_puntos, 2)

print("Haz clic en 4 puntos en el primer fotograma del VÍDEO (rojo).")
print("Luego haz clic en 4 puntos en la imagen DESTINO (azul).")

cv.imshow('Fuente (Video)', clone1)
cv.imshow('Destino', img2)

# Esperar selección de puntos
while True:
    if cv.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv.destroyAllWindows()
cv.waitKey(1)


Haz clic en 4 puntos en el primer fotograma del VÍDEO (rojo).
Luego haz clic en 4 puntos en la imagen DESTINO (azul).
Homografía calculada. Reproduciendo vídeo proyectado...


KeyboardInterrupt: 

In [ ]:
# Distorsión de barril y almohadilla en vídeo

import cv2
import numpy as np

video = cv.VideoCapture('videos/times-square.mp4')
video 

if not video.isOpened():
    print("Error: no se pudo abrir el video")
    exit()
    
ret, frame = video.read()
if not ret:
    print("Error: video vacío")
    exit()

size = (frame.shape[1], frame.shape[0])
w, h = size

def nothing(x):
    pass

def apply_distortion(image, k1, k2):
    h, w = image.shape[:2]
    
    distCoeff = np.zeros((4,1), np.float64)
    distCoeff[0,0] = k1  
    distCoeff[1,0] = k2  

    # Suponer matriz unitaria para la cámara
    cam = np.eye(3, dtype=np.float32)
    cam[0,2] = w/2.0  # Definir centro x
    cam[1,2] = h/2.0  # Definir centro y
    cam[0,0] = 10.    # Definir longitud focal x
    cam[1,1] = 10.    # Definir longitud focal y

    # Aplicar distorsión a la imagen
    distorted_img = cv2.undistort(image, cam, distCoeff)

    return distorted_img


cv2.namedWindow("Distorsion")

cv2.createTrackbar('Barril/Almohadilla', 'Distorsion', 0, 1, nothing)
cv2.createTrackbar('K1', 'Distorsion', 0, 10, nothing)
cv2.createTrackbar('K2', 'Distorsion', 0, 10, nothing)

while True:
    ret, frame = video.read()
    
    if not ret:
            video.set(cv.CAP_PROP_POS_FRAMES, 0)
            continue
    
    modo = cv2.getTrackbarPos('Barril/Almohadilla', 'Distorsion')
    k1 = cv2.getTrackbarPos('K1', 'Distorsion')
    k2 = cv2.getTrackbarPos('K2', 'Distorsion')
    
    if modo == 0:
        k1 = k1 / 100000
        k2 = k2 / 100000
    else:
        k1 = k1 / -100000
        k2 = k2 / -100000
    
    distorsion_vid = apply_distortion(frame, k1, k2)
    distorsion_resize = cv.resize(distorsion_vid, (800, 400))
    
    cv2.imshow("Distorsion", distorsion_resize)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
cv.waitKey(1)


-1

- Tratar una recta que será el eje de reflexión y “reflejar” la imagen.

In [36]:
import cv2 as cv
import numpy as np

img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

h, w = img.shape[:2]

# Variables globales
punto1 = None
punto2 = None
puntos_completos = False

def mouse_callback(event, x, y, flags, param):
    """Captura dos puntos para definir la línea de reflexión"""
    global punto1, punto2, puntos_completos

    if event == cv.EVENT_LBUTTONDOWN:
        if punto1 is None:
            punto1 = (x, y)
            print(f"Primer punto: {punto1}")
        elif punto2 is None:
            punto2 = (x, y)
            puntos_completos = True
            print(f"Segundo punto: {punto2}")

def reflejar_imagen(img, p1, p2):
    """Refleja la imagen respecto a la línea definida por p1 y p2"""
    h, w = img.shape[:2]
    x1, y1 = p1
    x2, y2 = p2

    dx = x2 - x1
    dy = y2 - y1
    length = np.sqrt(dx**2 + dy**2)
    if length == 0:
        return img, (0, 0), (w, h)

    dx /= length
    dy /= length

    cos_2theta = dx**2 - dy**2
    sin_2theta = 2 * dx * dy

    M_reflect = np.array([
        [cos_2theta, sin_2theta],
        [sin_2theta, -cos_2theta]
    ], dtype=np.float32)

    translation = np.array([x1, y1], dtype=np.float32)
    reflected_translation = translation - M_reflect @ translation
    M = np.hstack([M_reflect, reflected_translation.reshape(2, 1)])

    corners = np.array([[0, 0], [w, 0], [w, h], [0, h]], dtype=np.float32)
    corners_transformed = np.array([M_reflect @ (c - translation) + translation for c in corners])
    all_corners = np.vstack([corners, corners_transformed])
    min_x, min_y = all_corners.min(axis=0)
    max_x, max_y = all_corners.max(axis=0)

    new_w = int(max(max_x - min_x, w * 2))
    new_h = int(max(max_y - min_y, h * 2))
    M[0, 2] -= min_x
    M[1, 2] -= min_y

    reflected = cv.warpAffine(img, M, (new_w, new_h))
    offset = (int(min_x), int(min_y))
    return reflected, offset, (new_w, new_h)

# ---------------------------
# Ventana principal
# ---------------------------
cv.namedWindow("Original")
cv.setMouseCallback("Original", mouse_callback)

print("Haz clic en dos puntos para definir el eje de reflexión")
print("Presiona 'R' para reiniciar los puntos")
print("Presiona 'ESC' para salir")

while True:
    img_display = img.copy()

    # Dibujar los puntos seleccionados
    if punto1 is not None:
        cv.circle(img_display, punto1, 5, (0, 255, 0), -1)
    if punto2 is not None:
        cv.circle(img_display, punto2, 5, (0, 255, 0), -1)

    # Dibujar la línea si existen ambos puntos
    if punto1 is not None and punto2 is not None:
        dx = punto2[0] - punto1[0]
        dy = punto2[1] - punto1[1]
        length = np.sqrt(dx**2 + dy**2)
        if length > 0:
            dx /= length
            dy /= length
            extension = max(w, h) * 2
            p_start = (int(punto1[0] - dx * extension), int(punto1[1] - dy * extension))
            p_end = (int(punto1[0] + dx * extension), int(punto1[1] + dy * extension))
            cv.line(img_display, p_start, p_end, (255, 0, 255), 2)

    cv.putText(img_display, "Selecciona 2 puntos para el eje de reflexion",
               (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Mostrar imagen original
    cv.imshow('Original', img_display)

    # Mostrar la imagen reflejada en otra ventana (con la recta)
    if puntos_completos:
        reflected, offset, size = reflejar_imagen(img, punto1, punto2)

        # Ajustar los puntos para dibujar la misma línea en la imagen reflejada
        p1_adj = (punto1[0] - offset[0], punto1[1] - offset[1])
        p2_adj = (punto2[0] - offset[0], punto2[1] - offset[1])
        dx = p2_adj[0] - p1_adj[0]
        dy = p2_adj[1] - p1_adj[1]
        length = np.sqrt(dx**2 + dy**2)
        if length > 0:
            dx /= length
            dy /= length
            extension = max(size)
            p_start = (int(p1_adj[0] - dx * extension), int(p1_adj[1] - dy * extension))
            p_end = (int(p1_adj[0] + dx * extension), int(p1_adj[1] + dy * extension))
            cv.line(reflected, p_start, p_end, (255, 0, 255), 2)

        cv.putText(reflected, "Imagen reflejada", (10, 30),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv.imshow("Reflejada", reflected)

    key = cv.waitKey(1) & 0xFF
    if key == 27:  # ESC
        break
    elif key in (ord('r'), ord('R')):
        punto1 = None
        punto2 = None
        puntos_completos = False
        cv.destroyWindow("Reflejada")
        print("Puntos reiniciados")

cv.destroyAllWindows()
cv.waitKey(1)


Haz clic en dos puntos para definir el eje de reflexión
Presiona 'R' para reiniciar los puntos
Presiona 'ESC' para salir
Primer punto: (73, 174)
Segundo punto: (328, 174)
Puntos reiniciados
Primer punto: (209, 118)
Segundo punto: (212, 277)
Puntos reiniciados


-1

- Marcar el punto de giro con el ratón.

In [31]:
import cv2 as cv
import numpy as np

img = cv.imread('images/eii.png')
size = (img.shape[1], img.shape[0])
h, w = img.shape[:2]
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

def nothing(x):
    pass

# Inicializar el punto en el centro de la imagen
cx = w // 2
cy = h // 2

def point_turn(event, x, y, flags, param):
    global cx, cy
    
    if event == cv.EVENT_LBUTTONDOWN:
        cx, cy = x, y
        print(f"Nuevo punto de rotación: ({cx}, {cy})")


cv.namedWindow("Rotacion")
cv.setMouseCallback("Rotacion", point_turn)
cv.createTrackbar('Angulo', 'Rotacion', 0, 360, nothing)

while True:
    ang = cv.getTrackbarPos('Angulo', 'Rotacion')
    
    # Obtener la matriz de rotación correcta alrededor del punto (cx, cy)
    M = cv.getRotationMatrix2D((cx, cy), ang, 1.0)
    
    imageR = cv.warpAffine(img, M, size)
    
    # Marcar el punto de rotación
    cv.circle(imageR, (cx, cy), 5, (0, 255, 255), -1)
    cv.circle(imageR, (cx, cy), 8, (0, 0, 255), 2)
    
    info_text = f"Angulo: {ang} | Centro: ({cx}, {cy})"
    cv.putText(imageR, info_text, (10, 30), 
                cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    cv.imshow('Rotacion', imageR)

    key = cv.waitKey(1) & 0xFF
    if key == 27:  # ESC
        break
    elif key == ord('r') or key == ord('R'):
        cx = w // 2
        cy = h // 2
        print("Punto de rotación reiniciado al centro")

cv.destroyAllWindows()
cv.waitKey(1)

-1

- Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real.

In [2]:
#Hacer Traslaciones con el raton
import cv2 as cv
import numpy as np

img = cv.imread('images/eii.png')
size = (img.shape[1], img.shape[0])
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

def nothing(x):
    pass

h, w = img.shape[:2]

draw = False
ix = 0
iy = 0
tx = 0
ty = 0

def move_image(event, x, y, flags, param):
    global draw, ix, iy, tx, ty, img
    
    if event == cv.EVENT_LBUTTONDOWN:
        draw = True
        ix = x - tx
        iy = y - ty
        
    elif event == cv.EVENT_MOUSEMOVE:
        if draw:
            tx = x - ix
            ty = y - iy
        
    elif event == cv.EVENT_LBUTTONUP:
        draw = False


cv.namedWindow("Traslacion con el raton")
cv.setMouseCallback("Traslacion con el raton", move_image)

while True:
    T = np.float32([[1,0,tx],
                    [0,1,ty]])

    imageT = cv.warpAffine(img, T, (w, h))
    
    cv.imshow('Traslacion con el raton', imageT)

    if cv.waitKey(1) & 0xFF == 27:
        break
    elif cv.waitKey(1) & 0xFF == ord('r') or cv.waitKey(1) & 0xFF == ord('R'):
        tx = 0
        ty = 0

cv.destroyAllWindows()
cv.waitKey(1)


-1

- Transformación afín de una imagen mediante 3 puntos

In [34]:
# Transformación afín de una imagen mediante 3 puntos

import cv2 as cv
import numpy as np

# ------------------------------
# Cargar imagen original
# ------------------------------
img = cv.imread('images/eii.png')
if img is None:
    print("Error: no se pudo cargar la imagen")
    exit()

h, w = img.shape[:2]
img2 = np.ones((h, w, 3), dtype=np.uint8) * 255  # segunda ventana en blanco

# Copias para dibujar los puntos
clone1 = img.copy()
clone2 = img2.copy()

# Listas para guardar puntos
pts_src = []
pts_dst = []

# ------------------------------
# Callback del ratón para ambas ventanas
# ------------------------------
def seleccionar_puntos(event, x, y, flags, param):
    global pts_src, pts_dst, clone1, clone2

    # param = 1 → primera imagen (fuente)
    # param = 2 → segunda imagen (destino)
    if event == cv.EVENT_LBUTTONDOWN:
        if param == 1 and len(pts_src) < 3:
            pts_src.append((x, y))
            cv.circle(clone1, (x, y), 5, (0, 0, 255), -1)
            cv.putText(clone1, f"{len(pts_src)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            
            # Dibujar líneas entre puntos
            if len(pts_src) > 1:
                cv.line(clone1, pts_src[-2], pts_src[-1], (0, 255, 0), 2)
            if len(pts_src) == 3:
                cv.line(clone1, pts_src[-1], pts_src[0], (0, 255, 0), 2)
            
            cv.imshow('Original', clone1)
            
            if len(pts_src) == 3:
                print("3 puntos seleccionados en la imagen ORIGINAL")
                print("Ahora selecciona 3 puntos en la imagen DESTINO")

        elif param == 2 and len(pts_dst) < 3:
            pts_dst.append((x, y))
            cv.circle(clone2, (x, y), 5, (255, 0, 0), -1)
            cv.putText(clone2, f"{len(pts_dst)}", (x + 10, y - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
            
            # Dibujar líneas entre puntos
            if len(pts_dst) > 1:
                cv.line(clone2, pts_dst[-2], pts_dst[-1], (0, 255, 0), 2)
            if len(pts_dst) == 3:
                cv.line(clone2, pts_dst[-1], pts_dst[0], (0, 255, 0), 2)
            
            cv.imshow('Destino', clone2)

        # Si ya hay 3 puntos en cada imagen, aplicar transformación afín
        if len(pts_src) == 3 and len(pts_dst) == 3:
            transformar_imagen()

# ------------------------------
# Función para realizar la transformación afín
# ------------------------------
def transformar_imagen():
    global pts_src, pts_dst, img, img2
    
    # Calcular matriz de transformación afín (requiere exactamente 3 puntos)
    M = cv.getAffineTransform(np.float32(pts_src), np.float32(pts_dst))

    # Aplicar transformación a la imagen original
    transformada = cv.warpAffine(img, M, (img2.shape[1], img2.shape[0]))

    # Mostrar resultados
    cv.imshow('Imagen Original', img)
    cv.imshow('Imagen Transformada', transformada)

# ------------------------------
# Ventanas e interacción
# ------------------------------
cv.namedWindow('Original')
cv.namedWindow('Destino')

cv.setMouseCallback('Original', seleccionar_puntos, 1)
cv.setMouseCallback('Destino', seleccionar_puntos, 2)

print("=" * 60)
print("TRANSFORMACIÓN AFÍN MEDIANTE 3 PUNTOS")
print("=" * 60)
print("1. Haz clic en 3 puntos en la imagen ORIGINAL (puntos rojos)")
print("2. Luego haz clic en 3 puntos en la imagen DESTINO (puntos azules)")
print("\nLos puntos deben estar en el MISMO ORDEN en ambas imágenes")
print("\nPresiona 'R' para reiniciar")
print("Presiona 'ESC' para salir")
print("=" * 60)

cv.imshow('Original', clone1)
cv.imshow('Destino', clone2)

# ------------------------------
# Loop principal
# ------------------------------
while True:
    key = cv.waitKey(1) & 0xFF
    
    if key == 27:  # ESC
        break
    elif key == ord('r') or key == ord('R'):  # Reiniciar
        pts_src = []
        pts_dst = []
        clone1 = img.copy()
        clone2 = img2.copy()
        cv.imshow('Original', clone1)
        cv.imshow('Destino', clone2)
        # Cerrar ventanas de resultado si existen
        cv.destroyWindow('Imagen Original')
        cv.destroyWindow('Imagen Transformada')

cv.destroyAllWindows()
cv.waitKey(1)

TRANSFORMACIÓN AFÍN MEDIANTE 3 PUNTOS
1. Haz clic en 3 puntos en la imagen ORIGINAL (puntos rojos)
2. Luego haz clic en 3 puntos en la imagen DESTINO (puntos azules)

Los puntos deben estar en el MISMO ORDEN en ambas imágenes

Presiona 'R' para reiniciar
Presiona 'ESC' para salir


-1

- Imagen especular

In [37]:
import cv2 as cv

img = cv.imread('images/eii.png')
if img is None:
    raise SystemExit("No se pudo cargar la imagen")

# Espejo horizontal (respecto al eje vertical): flipCode = 1
mirror_h = cv.flip(img, 1)

cv.imshow('Original', img)
cv.imshow('Imagen especular', mirror_h)


cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)  # cierre robusto de ventanas


2025-10-27 17:46:35.124 Python[87960:4909301] _mthid_copyDeviceInfo(288230376418898271) failed


-1

# Cosas por hacer:  
- Otras aportaciones

In [ ]:
import cv2 as cv
import numpy as np

# Variables globales
scale = 1.0
center_x, center_y = 0, 0
img_original = None
img_display = None
mouse_x, mouse_y = 0, 0

def aplicar_zoom(img, scale, center_x, center_y):
    """Aplica zoom a la imagen usando el centro especificado"""
    h, w = img.shape[:2]
    
    # Crear matriz de rotación con escala (rotación = 0)
    M = cv.getRotationMatrix2D((center_x, center_y), 0, scale)
    
    # Aplicar transformación
    img_zoom = cv.warpAffine(img, M, (w, h), flags=cv.INTER_LINEAR)
    
    return img_zoom

def mouse_callback(event, x, y, flags, param):
    """Callback para eventos del ratón"""
    global scale, center_x, center_y, img_original, img_display, mouse_x, mouse_y
    
    # Actualizar posición del ratón
    if event == cv.EVENT_MOUSEMOVE:
        mouse_x, mouse_y = x, y
    
    # Zoom con rueda del ratón (funciona en MacBook trackpad con scroll)
    if event == cv.EVENT_MOUSEWHEEL:
        # Actualizar centro del zoom a la posición actual del ratón
        center_x, center_y = x, y
        
        # En MacOS, flags contiene el delta del scroll
        # Positivo = scroll hacia arriba (zoom in)
        # Negativo = scroll hacia abajo (zoom out)
        if flags > 0:
            scale *= 1.15
        else:
            scale *= 0.85
        
        # Limitar escala entre 0.3 y 10.0
        scale = max(0.3, min(scale, 10.0))
        
        # Aplicar zoom
        img_display = aplicar_zoom(img_original, scale, center_x, center_y)
        actualizar_display()

def actualizar_display():
    """Actualiza la visualización con información"""
    global img_display, scale, center_x, center_y, mouse_x, mouse_y
    
    img_info = img_display.copy()
    
    # Información de zoom
    texto_zoom = f"Zoom: {scale:.2f}x"
    cv.putText(img_info, texto_zoom, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 
               0.8, (0, 255, 0), 2, cv.LINE_AA)
    
    # Información de centro
    texto_centro = f"Centro: ({center_x}, {center_y})"
    cv.putText(img_info, texto_centro, (10, 60), cv.FONT_HERSHEY_SIMPLEX, 
               0.6, (0, 255, 0), 2, cv.LINE_AA)
    
    # Instrucciones
    texto_inst = "Trackpad: 2 dedos arriba/abajo | Teclado: +/- | R: Reset | ESC: Salir"
    cv.putText(img_info, texto_inst, (10, img_info.shape[0] - 15), 
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv.LINE_AA)
    
    # Dibujar cruz en el centro del zoom
    cv.drawMarker(img_info, (center_x, center_y), (0, 255, 0), 
                  cv.MARKER_CROSS, 25, 2)
    
    # Dibujar círculo en posición del ratón
    cv.circle(img_info, (mouse_x, mouse_y), 5, (255, 0, 255), -1)
    
    cv.imshow('Zoom Interactivo - MacBook', img_info)

# Cargar imagen de ejemplo
img_original = np.zeros((600, 800, 3), dtype=np.uint8)

# Crear un patrón de ejemplo más detallado
# Fondo degradado
for i in range(600):
    color = int(30 + (i / 600) * 50)
    cv.line(img_original, (0, i), (800, i), (color, color, color), 1)

# Grid más fino para ver el zoom en detalle
for i in range(0, 800, 25):
    color = (80, 80, 80) if i % 50 == 0 else (60, 60, 60)
    grosor = 2 if i % 50 == 0 else 1
    cv.line(img_original, (i, 0), (i, 600), color, grosor)
for i in range(0, 600, 25):
    color = (80, 80, 80) if i % 50 == 0 else (60, 60, 60)
    grosor = 2 if i % 50 == 0 else 1
    cv.line(img_original, (0, i), (800, i), color, grosor)

# Añadir círculos de colores con gradientes
colores = [(255, 50, 50), (50, 255, 50), (50, 50, 255), 
           (255, 255, 50), (255, 50, 255), (50, 255, 255)]
for i, color in enumerate(colores):
    x = 150 + (i % 3) * 250
    y = 150 + (i // 3) * 300
    # Círculo exterior
    cv.circle(img_original, (x, y), 90, color, -1)
    # Círculo medio
    cv.circle(img_original, (x, y), 60, tuple([c//2 for c in color]), -1)
    # Círculo interior
    cv.circle(img_original, (x, y), 30, (255, 255, 255), -1)
    # Número
    cv.putText(img_original, str(i+1), (x-15, y+15), 
               cv.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3, cv.LINE_AA)

# Añadir texto con sombra
texto = 'Zoom MacBook Trackpad'
cv.putText(img_original, texto, (152, 52), 
           cv.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 5, cv.LINE_AA)
cv.putText(img_original, texto, (150, 50), 
           cv.FONT_HERSHEY_SIMPLEX, 1.3, (255, 255, 255), 3, cv.LINE_AA)

# Si quieres cargar tu propia imagen:
# img_original = cv.imread('tu_imagen.jpg')

if img_original is None:
    print("Error: No se pudo cargar la imagen")
    exit()

# Inicializar centro en el medio de la imagen
h, w = img_original.shape[:2]
center_x, center_y = w // 2, h // 2
mouse_x, mouse_y = w // 2, h // 2

img_display = img_original.copy()

# Crear ventana y asignar callback
cv.namedWindow('Zoom Interactivo - MacBook')
cv.setMouseCallback('Zoom Interactivo - MacBook', mouse_callback)

# Mostrar instrucciones
print("=" * 60)
print("ZOOM INTERACTIVO PARA MACBOOK (Trackpad compatible)")
print("=" * 60)
print("Instrucciones:")
print("  TRACKPAD:")
print("    - Desliza 2 dedos ARRIBA para ACERCAR (zoom in)")
print("    - Desliza 2 dedos ABAJO para ALEJAR (zoom out)")
print("    - El zoom se centra donde está el cursor")
print("")
print("  TECLADO (alternativa):")
print("    - Tecla '+' o '=' para acercar")
print("    - Tecla '-' para alejar")
print("    - Tecla 'R' para resetear el zoom")
print("    - Tecla 'ESC' o 'Q' para salir")
print("=" * 60)

# Bucle principal
while True:
    actualizar_display()
    
    # Esperar por tecla
    key = cv.waitKey(10) & 0xFF
    
    if key == 27 or key == ord('q'):  # ESC o Q para salir
        break
    elif key == ord('r'):  # R para resetear
        scale = 1.0
        center_x, center_y = w // 2, h // 2
        img_display = img_original.copy()
        print("✓ Zoom reseteado")
    elif key == ord('+') or key == ord('='):  # + para zoom in
        center_x, center_y = mouse_x, mouse_y
        scale *= 1.15
        scale = min(scale, 10.0)
        img_display = aplicar_zoom(img_original, scale, center_x, center_y)
        print(f"✓ Zoom: {scale:.2f}x")
    elif key == ord('-'):  # - para zoom out
        center_x, center_y = mouse_x, mouse_y
        scale *= 0.85
        scale = max(scale, 0.3)
        img_display = aplicar_zoom(img_original, scale, center_x, center_y)
        print(f"✓ Zoom: {scale:.2f}x")

cv.destroyAllWindows()
print("\n¡Programa finalizado! 👋")
cv.waitKey(1)  # cierre robusto de ventanas


ZOOM INTERACTIVO PARA MACBOOK (Trackpad compatible)
Instrucciones:
  TRACKPAD:
    - Desliza 2 dedos ARRIBA para ACERCAR (zoom in)
    - Desliza 2 dedos ABAJO para ALEJAR (zoom out)
    - El zoom se centra donde está el cursor

  TECLADO (alternativa):
    - Tecla '+' o '=' para acercar
    - Tecla '-' para alejar
    - Tecla 'R' para resetear el zoom
    - Tecla 'ESC' o 'Q' para salir
✓ Zoom reseteado
✓ Zoom reseteado
✓ Zoom reseteado
✓ Zoom reseteado
✓ Zoom: 1.15x
✓ Zoom: 1.32x
✓ Zoom: 1.52x
✓ Zoom: 1.29x
✓ Zoom: 1.10x
✓ Zoom: 0.93x
✓ Zoom: 0.79x
✓ Zoom: 0.67x
✓ Zoom: 0.57x
✓ Zoom: 0.49x
✓ Zoom: 0.56x
✓ Zoom: 0.64x
✓ Zoom: 0.74x
✓ Zoom: 0.85x
✓ Zoom: 0.98x
✓ Zoom: 1.13x
✓ Zoom: 1.30x
✓ Zoom: 1.49x
✓ Zoom: 1.72x
✓ Zoom: 1.97x
✓ Zoom: 2.27x
✓ Zoom: 1.93x
✓ Zoom: 1.64x
✓ Zoom: 1.39x
✓ Zoom: 1.18x
